In [1]:
## Load the packages we will use:
# -------------------
using LinearAlgebra
using Plots
using CSV, DataFrames
using ForwardDiff
using Interpolations, Polynomials

# Helper Functions

In [10]:
function decompose(lambda)
    n = size(lambda)[1] - 2
    a = [lambda[i,i+1] for i = 2:n+1]
    b = [lambda[i,i-1] for i = 3:n+1]
    c = [lambda[i,n+2] / lambda[1,n+2] for i = 2:n]
    d = [lambda[1,i] for i = 2:n+2]

    return a, b, c, d
end

function recompose(a,b,c,d)
    n = length(a)
    lambda = zeros(eltype(c), n+2,n+2)
    for i = 1:n
        lambda[i+1,i+2] = a[i]
    end
    for i = 1:n-1
        lambda[i+2, i+1] = b[i]
    end
    for i = 1:(n+1)
        lambda[1, i+1] = c[i]
    end

    for i = 1:n-1
        for j = i+2:(n+1)
            lambda[i+1,j+1] = d[i] * c[j]
        end
    end
    return lambda
end

function compute_eta(N)
    low = 1.0
    high = 2.0
    for i in 1:50
        mid = (low+high)/2.0
        val = (1+2*N*mid)*(mid-1)^(2*N)
        if val <1
            low = mid
        else
            high =mid
        end
    end
    return low
end

function compute_parts_from_d(eta, r, N, d)
    a = zeros(eltype(d), N)
    b = zeros(eltype(d), N-1)
    c = zeros(eltype(d), N+1)
    c[N+1] = sqrt(2*r)
    c[N] = 2*r*(1+sum(d)+(eta-1)/sqrt(2*r))
    for i in 1:(N-1)
        c[i] = 2*r*(eta*sum(d[1:i]) - d[i]+eta)
    end
    
    a[N] = 1 - sqrt(2*r)*(1+sum(d))
    a[N-1] = 1/eta * (c[N]^2/(2*r) -a[N] - (1+eta)*c[N]*(1+sum(d[1:(N-2)])) + c[N-1]*c[N]/(2*r))
    b[N-1] = 1/eta * ((eta-1)*c[N]^2/(2*r) +(1-eta)*a[N] + c[N]*(1+sum(d[1:(N-2)])) - c[N-1]*c[N]/(2*r))
    for w in 1:(N-3)
        i = N-1-w
        a[i] = 1/eta * (c[i+1]^2/(2*r) + c[i]*c[i+1]/(2*r) - a[i+1] - d[i+1]*(sum(c[(i+3):(N+1)])) -(1+eta)*c[i+1]*(1+sum(d[1:(i-1)])) +b[i+1]*(2*eta-1))
        b[i] = 1/eta * ((eta-1)*c[i+1]^2/(2*r) - c[i]*c[i+1]/(2*r) - (eta-1)*a[i+1] - (eta-1)*d[i+1]*(sum(c[(i+3):(N+1)])) + c[i+1]*(1+sum(d[1:(i-1)])) +(eta-1)*b[i+1]*(2*eta-1))
    end
    a[1] = 1/eta * (c[2]^2/(2*r) + c[1]*c[2]/(2*r) -a[2] - (1+eta)*c[2] -d[2]*sum(c[4:(N+1)]) + b[2]*(2*eta-1))
    b[1] = 1/eta * ((eta-1)*c[2]^2/(2*r) - c[1]*c[2]/(2*r) -(eta-1)*a[2] - (eta-1)*d[2]*sum(c[4:(N+1)]) + c[2] + (eta-1)*b[2]*(2*eta-1))
    return a,b,c
end

function rowSumMinusColumnSumMinusTarget(a,b,c,d,n)
    #Compute the difference of row sums and column sums from the goal
    eta = compute_eta(n)
    r = 1 / (2 * (1 + 2 * n * eta))
    target = zeros(eltype(d),n+2)
    target[1]=-1
    target[n+2]=1
    
    target[2:(n+1)] += a[1:n]
    target[3:(n+2)] -= a[1:n]

    target[3:(n+1)] += b[1:(n-1)]
    target[2:n] -= b[1:(n-1)]

    target[1] += sum(c[1:(n+1)])
    target[2:(n+2)] -= c[1:(n+1)]

    for i = 1:n-1
        target[i+1] +=  d[i] * sum(c[i+2:(n+1)])
        target[i+3:(n+2)] -= d[i] * c[i+2:(n+1)]
    end
    #First and last equality are already enforced. They will equal zero for free.
    
    #We can replace first one by the final equality needed for the coefficient of ||g_0|| to be zero
    target[1] = -c[1] - a[1] - d[1]*sum(c[3:(n+1)]) + (2*eta-1)*b[1] + c[1]^2/(2*r)
    return target[1:(n+1)] 
end

function linear_feasible(d)
    n = length(d)+1
    eta = compute_eta(n)
    r = 1 / (2 * (1 + 2 * n * eta))
    a,b,c = compute_parts_from_d(eta,r,n,d)
    
    return rowSumMinusColumnSumMinusTarget(a,b,c,d,n) # lambda*ones(n+2) - lambda'*ones(n+2) - target
end

function newtonStep(d)
    x = d
    n = length(d) + 1
    J = zeros(n+1, n-1)
    for i in 1:7
        F = linear_feasible(x)
        if norm(F)<1e-10
            println("    ",i)
            break
        end
        #J = finite_difference_jacobian(linear_feasible, x)
        ForwardDiff.jacobian!(J, linear_feasible, x) #finite_difference_jacobian(linear_feasible, x)
        Δx = -J \ F  # Solve the linear system J * Δx = -F
        x += Δx      # Update the guess
        println("   Newton Iteration $i: Solution guess: ", norm(F))
    end
    return x
end

function guessNext(d,n) #A simple heuristic for guessing the next solution
    d_next = zeros(n)
    d_next[2:n] = d
    d_next[1] = d[1]
    return d_next
end

function guessNextHistory(d1,d2,d3,d4,d5,step)
    #Guess c0 from the pattern
    int1 = Interpolations.scale(interpolate(d1, BSpline(Quadratic(Line(OnCell())))), 0:(1.0/(length(d1)-1)):1)
    int2 = Interpolations.scale(interpolate(d2, BSpline(Quadratic(Line(OnCell())))), 0:(1.0/(length(d2)-1)):1)
    int3 = Interpolations.scale(interpolate(d3, BSpline(Quadratic(Line(OnCell())))), 0:(1.0/(length(d3)-1)):1)
    int4 = Interpolations.scale(interpolate(d4, BSpline(Quadratic(Line(OnCell())))), 0:(1.0/(length(d4)-1)):1)
    int5 = Interpolations.scale(interpolate(d5, BSpline(Quadratic(Line(OnCell())))), 0:(1.0/(length(d5)-1)):1)
    
    n = length(d1) + step
    d_next = zeros(n)
    for i in 1:n
        t = [1,2,3,4,5]
        s = [int1((i-1.0)/(n-1.0)), int2((i-1.0)/(n-1.0)),int3((i-1.0)/(n-1.0)),int4((i-1.0)/(n-1.0)),int5((i-1.0)/(n-1.0))] 
        p = Polynomials.fit(t, s, 1)
        d_next[i] = max(p(0),0)
    end
    return d_next
end

function finite_difference_jacobian(f, x, epsilon=1e-8) #ForawrdDiff wasnt playing nice with offset arrays
    n = length(x)
    m = length(f(x))
    J = zeros(m, n)

    # Iterate over each input dimension to perturb
    for j in 1:n
        println("       FD: ",j)
        x_forward = copy(x)
        x_backward = copy(x)

        # Perturb the j-th element
        x_forward[j] += epsilon
        x_backward[j] -= epsilon

        # Compute the function values with perturbed inputs
        f_forward = f(x_forward)
        f_backward = f(x_backward)

        # Compute the finite difference approximation for column j
        J[:, j] = (f_forward - f_backward) / (2 * epsilon)
    end

    return J
end
    
function postiveSum(v)
    ret = 0
    for val in v
        ret += max(0,val)
    end
    return ret
end

postiveSum (generic function with 1 method)

# Verify Existing Certificates

In [11]:
println("Verifing Certificates for n=3,4,5,...,2240")
deltaMaximum = 0.0
for n in 3:2200
    eta = compute_eta(n)
    r = 1 / (2 * (1 + 2 * n * eta))
    d = CSV.read(string(n,"c.csv"), DataFrame).Column
    a,b,c = compute_parts_from_d(eta, r, n, d)
    lambda = recompose(a,b,c,d)
    if minimum(lambda)<0
        println("INFEASIBLE CERTIFICATE: ",n)
    end
    deltaMaximum = postiveSum(linear_feasible(d))
end
println("    Maximum delta: ",deltaMaximum)

Verifing Certificates for n=3,4,5,...,2240
    Maximum delta: 1.7540900372828294e-14


In [12]:
println("Verifing Certificates for n=2240,2560,...,8960")
deltaMaximum = 0.0
for n in 2240:320:8960
    eta = compute_eta(n)
    r = 1 / (2 * (1 + 2 * n * eta))
    d = CSV.read(string(n,"c.csv"), DataFrame).Column
    a,b,c = compute_parts_from_d(eta, r, n, d)
    lambda = recompose(a,b,c,d)
    if minimum(lambda)<0
        println("INFEASIBLE CERTIFICATE: ",n)
    end
    deltaMaximum = postiveSum(linear_feasible(d))
end
println("    Maximum delta: ",deltaMaximum)

Verifing Certificates for n=2240,2560,...,8960
    Maximum delta: 5.77358053401901e-14


In [13]:
println("Verifing Certificates for n=8960,10560,...,18560")
deltaMaximum = 0.0
for n in 8960:1600:18560
    eta = compute_eta(n)
    r = 1 / (2 * (1 + 2 * n * eta))
    d = CSV.read(string(n,"c.csv"), DataFrame).Column
    a,b,c = compute_parts_from_d(eta, r, n, d)
    lambda = recompose(a,b,c,d)
    if minimum(lambda)<0
        println("INFEASIBLE CERTIFICATE: ",n)
    end
    deltaMaximum = postiveSum(linear_feasible(d))
end
println("    Maximum delta: ",deltaMaximum)

Verifing Certificates for n=8960,10560,...,18560
    Maximum delta: 1.076808015313107e-13


# Generate New Certificates via Newton's Method

In [ ]:
n = 7
eta = compute_eta(n)
r = 1 / (2 * (1 + 2 * n * eta))

stepSize = 1
x_olderby5 = CSV.read(string(n-4*stepSize,"c.csv"), DataFrame).Column
x_olderby4 = CSV.read(string(n-3*stepSize,"c.csv"), DataFrame).Column
x_olderby3 = CSV.read(string(n-2*stepSize,"c.csv"), DataFrame).Column
x_olderby2 = CSV.read(string(n-1*stepSize,"c.csv"), DataFrame).Column
x = CSV.read(string(n,"c.csv"), DataFrame).Column

start_time = time()

for t in 8:stepSize:256
    a,b,c = compute_parts_from_d(eta, r, n, x)
    lambda = recompose(a,b,c,x)
    println("Certicate ",n,"  with validation of ", minimum(lambda)>=0, " and ", norm(linear_feasible(x))<=1e-10, ".   its been ", round((time()-start_time)/3600, digits=5), " hours")

    n=t
    xNext = guessNextHistory(x,x_olderby2,x_olderby3,x_olderby4,x_olderby5,stepSize)
    eta = compute_eta(n)
    r = 1 / (2 * (1 + 2 * n * eta))
    xNext = newtonStep(xNext)
    
    df = DataFrame(Column = xNext)
    CSV.write(string(n,"c.csv"), df)
    
    x_olderby5 = x_olderby4
    x_olderby4 = x_olderby3
    x_olderby3 = x_olderby2
    x_olderby2 = x
    x = xNext
end

3200 Starting Loop
3200  with validation of true and true.   its been 0.00018 hours
    Iteration 1: Solution guess: 0.07600946106572032
    Iteration 2: Solution guess: 0.0015195969226130628
    Iteration 3: Solution guess: 2.380594173059902e-6
    4
3520 Ending Loop
3520 Starting Loop
3520  with validation of true and true.   its been 0.2561 hours
    Iteration 1: Solution guess: 0.0595588844124949
    Iteration 2: Solution guess: 0.0009210256372086486
    Iteration 3: Solution guess: 8.648117260337487e-7
    4
3840 Ending Loop
3840 Starting Loop
3840  with validation of true and true.   its been 0.59517 hours
    Iteration 1: Solution guess: 0.04794418607839222
    Iteration 2: Solution guess: 0.0005924231660197825
    Iteration 3: Solution guess: 3.549383592582388e-7
    4
4160 Ending Loop
4160 Starting Loop
4160  with validation of true and true.   its been 1.05281 hours
    Iteration 1: Solution guess: 0.03943347266036853
    Iteration 2: Solution guess: 0.00039905429209406175
  